In [1]:
import numpy as np
import pandas as pd
import scipy.io as io
import os

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor

torch.cuda.is_available()

True

In [3]:
import timm
from torch.utils.data.sampler import SubsetRandomSampler, WeightedRandomSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


In [4]:
import wandb
# WANDB_NOTEBOOK_NAME = 'Train_Analysis_VRB_wandb_sweep.ipynb'
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: xiaosuhu86. Use `wandb login --relogin` to force relogin


True

In [5]:
sweep_configuration = {
    'method': 'grid',
    'name': 'sweep',
    'metric': {'goal': 'minimize', 'name': 'val_loss'},
    'parameters': 
    {
        #'batch_size': {'values': [32, 64]},
        #'epochs': {'values': [5,10]},
        #'lr': {'values': [0.001, 0.0001]},
        'batch_size': {'values': [32]},
        'epochs': {'values': [2]},
        'lr': {'values': [0.001]},
        'model': {'values': ['efficientnet_b0','efficientnet_b2']}
     }
}

sweep_id = wandb.sweep(sweep=sweep_configuration, project='VRB-Pytorch-test')


Create sweep with ID: 96h6g0gx
Sweep URL: https://wandb.ai/xiaosuhu86/VRB-Pytorch-test/sweeps/96h6g0gx


In [6]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [7]:
class FNIRS_Dataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        self.targets = self.img_labels['Label']

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 4])
        image = np.float32(io.loadmat(img_path)['fnirsimg']).reshape(3,21,45)
        label = self.img_labels.iloc[idx, 5]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    return loss

def validate(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    model.eval()     # Optional when not using Model Specific layer

    total_correct = 0
    total_instances = 0
    y_true=[]
    y_pred=[]

    for batch, (X,y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred,y)
        classifications = torch.argmax(model(X), dim=1)
        correct_predictions = sum(classifications==y).item()

        total_correct+=correct_predictions
        total_instances+=len(y)

        y_pred.extend(classifications.data.cpu().numpy())
        y_true.extend(y.data.cpu().numpy())
            
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"val_loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    
    acc = total_correct/total_instances

    # constant for classes
    classes = ('No_Pain','Pain')
    cf_matrix = confusion_matrix(y_true, y_pred)
    df_cm = pd.DataFrame(cf_matrix / np.sum(cf_matrix, axis=1)[:, None], index = [i for i in classes],
                     columns = [i for i in classes])
    
    TP = df_cm['Pain']['Pain']
    TN = df_cm['No_Pain']['No_Pain']

    print(f"val_acc: {acc:>7f} TP: {TP:>4f} TN: {TN:>4f}")

    return loss, acc, TP, TN

In [9]:
# Load the data with imbalanced weights
train_data=FNIRS_Dataset(
    '../Label_VRB.csv',
    '../VRBdata/'
)

total_targets=torch.asarray(train_data.targets)
train_idx, valid_idx= train_test_split(
    np.arange(len(total_targets)), test_size=0.2, random_state=42, shuffle=True, stratify=total_targets)

train_set = torch.utils.data.Subset(train_data, train_idx)
# val_set = torch.utils.data.Subset(train_data, valid_idx)

train_sample_count = torch.tensor(
    [(total_targets[train_idx] == t).sum() for t in torch.unique(total_targets, sorted=True)])
train_weight = 1. / train_sample_count.float()
train_sample_weight = torch.tensor([train_weight[t] for t in total_targets[train_idx]])

#Creating PT data samplers
train_sampler = WeightedRandomSampler(train_sample_weight, len(train_sample_weight))
valid_sampler = SubsetRandomSampler(valid_idx)

def main():
    run = wandb.init()

    batch_size = wandb.config.batch_size

    # Create data loaders:
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, 
                                            sampler=train_sampler)
    validation_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                                    sampler=valid_sampler)

    # Model defining
    model = timm.create_model(wandb.config.model, num_classes=2, pretrained=False)
    model=model.to(device)
    
    # Other learning parameters
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=wandb.config.lr)
    epochs = wandb.config.epochs

    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train_loss = train(train_loader, model, loss_fn, optimizer)
        val_loss, val_acc, TP, TN = validate(validation_loader, model, loss_fn)

        wandb.log({
        'epoch': t+1, 
        #'train_acc': train_acc,
        'train_loss': train_loss, 
        'val_acc': val_acc, 
        'val_loss': val_loss,
        'true_pos': TP,
        'true_neg': TN,
        'model': wandb.config.model
        })

wandb.agent(sweep_id, function=main)

wandb: Agent Starting Run: weovikiy with config:
wandb: 	batch_size: 32
wandb: 	epochs: 2
wandb: 	lr: 0.001
wandb: 	model: efficientnet_b0
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1
-------------------------------
loss: 3.721677  [   32/11536]
loss: 0.790990  [ 3232/11536]
loss: 1.106159  [ 6432/11536]
loss: 1.177879  [ 9632/11536]
val_loss: 0.430236  [   32/14420]
val_acc: 0.834605 TP: 0.074499 TN: 0.939250
Epoch 2
-------------------------------
loss: 0.733203  [   32/11536]
loss: 0.874047  [ 3232/11536]
loss: 0.791018  [ 6432/11536]
loss: 0.884339  [ 9632/11536]
val_loss: 0.782418  [   32/14420]
val_acc: 0.492718 TP: 0.601719 TN: 0.477712


epoch,▁█
train_loss,█▁
true_neg,█▁
true_pos,▁█
val_acc,█▁
val_loss,▁█
epoch,2
model,efficientnet_b0
train_loss,0.82395
true_neg,0.47771
true_pos,0.60172


wandb: Agent Starting Run: ouavjsp2 with config:
wandb: 	batch_size: 32
wandb: 	epochs: 2
wandb: 	lr: 0.001
wandb: 	model: efficientnet_b2
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1
-------------------------------
loss: 3.227458  [   32/11536]
loss: 1.052659  [ 3232/11536]
loss: 0.799534  [ 6432/11536]
loss: 0.815652  [ 9632/11536]
val_loss: 0.800530  [   32/14420]
val_acc: 0.252774 TP: 0.836676 TN: 0.172387
Epoch 2
-------------------------------
loss: 0.979755  [   32/11536]
loss: 0.662098  [ 3232/11536]
loss: 0.669749  [ 6432/11536]
loss: 0.674141  [ 9632/11536]
val_loss: 0.460959  [   32/14420]
val_acc: 0.825589 TP: 0.060172 TN: 0.930966


epoch,▁█
train_loss,█▁
true_neg,▁█
true_pos,█▁
val_acc,▁█
val_loss,█▁
epoch,2
model,efficientnet_b2
train_loss,0.60923
true_neg,0.93097
true_pos,0.06017


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
